## clean data

In [1]:
import pandas as pd
import itertools
from collections import Counter

In [2]:
pwd

u'/Users/XIANFEI/Downloads/SVM'

In [3]:
data = pd.read_csv('./preprocesse_samples_from_first_100000.csv',sep=',',quotechar='|',header=None)
data.columns = ['title','body','tags']

In [4]:
data.head(3)

,title,body,tags
0,While applying opacity to a form should we use...,I want to use a track-bar to change a form's o...,<c#><winforms><type-conversion><decimal><opacity>
1,Percentage width child element in absolutely p...,I have an absolutely positioned div containing...,<html><css><css3><internet-explorer-7>
2,How do I calculate someone's age in C#?,Given a DateTime representing a person's birth...,<c#><.net><datetime>


In [5]:
all_tags = data.tags.apply(lambda x: x.replace('<','').split('>')[:-1])
data_tags = all_tags
all_tags = [x for x in list(itertools.chain(*all_tags)) if x ]

In [6]:
ct = Counter(all_tags)
vocab = ct.most_common()

In [7]:
pd_tag=pd.DataFrame(vocab)

In [8]:
pd_tag.columns=['name','count']

In [9]:
data_tags

0        [c#, winforms, type-conversion, decimal, opacity]
1                   [html, css, css3, internet-explorer-7]
2                                     [c#, .net, datetime]
3             [c#, datetime, datediff, relative-time-span]
4        [javascript, html, browser, timezone, timezone...
5                                             [.net, math]
6                       [c#, linq, web-services, .net-3.5]
7             [mysql, database, binary-data, data-storage]
8        [performance, algorithm, language-agnostic, un...
9                              [mysql, database, triggers]
10                       [c++, c, sockets, mainframe, zos]
11                       [actionscript-3, flex, bytearray]
12                     [sql, sql-server, datatable, rdbms]
13                               [c#, .net, vb.net, timer]
14                      [php, plugins, architecture, hook]
15           [html, form-submit, html-form, submit-button]
16                                    [c#, linq, .net-3.

In [11]:
tags_list = pd_tag[pd_tag['count']>=300] #
tag_used = [ x for x in tags_list.name]
tag_used
tag_result = []
for tags in data_tags:
    tag_result.append([x for x in tags if x in tag_used])
tag_result

[['c#', 'winforms'],
 ['html', 'css'],
 ['c#', '.net'],
 ['c#'],
 ['javascript', 'html'],
 ['.net'],
 ['c#'],
 ['mysql', 'database'],
 ['performance'],
 ['mysql', 'database'],
 ['c++', 'c'],
 [],
 ['sql', 'sql-server'],
 ['c#', '.net'],
 ['php'],
 ['html'],
 ['c#'],
 [],
 [],
 ['ruby'],
 ['svn'],
 [],
 ['php', 'sql', 'database'],
 ['linux'],
 ['svn'],
 ['.net', 'performance'],
 ['windows', 'svn'],
 ['c#', 'sql'],
 ['sql', 'asp.net'],
 ['java', 'xml'],
 ['java', 'php'],
 ['mysql', 'sql-server'],
 ['.net', 'database'],
 ['c#', '.net'],
 ['php'],
 ['.net', 'asp.net', 'svn'],
 ['windows'],
 ['sql-server', 'database', 'svn'],
 ['c#', 'html'],
 [],
 ['linux'],
 [],
 ['c#', '.net', 'asp.net'],
 ['sql-server'],
 ['php'],
 ['windows'],
 ['mysql', 'sql-server'],
 ['.net', 'database'],
 ['c#', '.net'],
 ['c', 'linux'],
 ['c++'],
 ['svn'],
 ['c#'],
 ['sql', 'database'],
 ['php'],
 ['c++'],
 [],
 ['python', 'xml'],
 [],
 [],
 ['java'],
 [],
 [],
 [],
 [],
 ['ruby'],
 ['python'],
 ['.net'],
 ['c#', 

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(tag_result)

In [13]:
import numpy as np
np.shape(Y)

(18820, 23)

In [14]:
#import nltk
#nltk.download()

In [15]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
stop = stopwords.words('english')
stop.extend(['want', 'to', "'s"])
english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', "'", "'m", "-","n't"]
stop.extend(english_punctuations)
to_be_removed = ["''", '``', '...', "'d", '2', '--', '/' , '>', '3', '=', 'x', 'e.g', '2005', '2008', '{', '}', '2.0', '0']
stop.extend(to_be_removed)




def Preprocess(text):
    return text.lower()

def Token(text):
    tokens = word_tokenize(text)
    texts_filtered = [stemmer.stem(word) for word in tokens]
    final_res = [word for word in texts_filtered if not word in stop]
    return final_res
    

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(preprocessor=Preprocess, tokenizer=Token, max_features=100)
X = vectorizer.fit_transform(data.body)

In [17]:
X

<18820x100 sparse matrix of type '<type 'numpy.float64'>'
	with 202980 stored elements in Compressed Sparse Row format>

## svm 

In [18]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(SVC(class_weight='balanced'))
clf.fit(X, Y)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1)

In [19]:
pre = clf.predict(X)

In [20]:
pre

array([[1, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 0, 1]])

In [21]:
mlb.inverse_transform(pre)

[('.net', 'asp.net', 'c#', 'svn', 'visual-studio', 'winforms'),
 ('css', 'html', 'javascript', 'mysql', 'php', 'python', 'ruby', 'xml'),
 ('python', 'ruby'),
 ('c', 'c++', 'mysql', 'performance', 'python'),
 ('asp.net', 'html', 'javascript', 'php'),
 ('.net', 'c#', 'performance', 'winforms', 'xml'),
 ('.net', 'asp.net', 'visual-studio'),
 ('database', 'mysql', 'performance', 'python', 'xml'),
 ('c++', 'linux', 'performance', 'visual-studio'),
 ('database', 'html', 'mysql', 'performance', 'ruby', 'sql', 'sql-server'),
 ('c', 'c#', 'c++', 'php', 'visual-studio', 'xml'),
 ('css', 'html', 'javascript', 'linux', 'python', 'ruby'),
 ('database', 'mysql', 'performance', 'sql', 'sql-server'),
 ('.net', 'asp.net', 'c#', 'winforms', 'xml'),
 ('asp.net', 'performance', 'php', 'python'),
 ('asp.net', 'css', 'html', 'javascript', 'php', 'winforms'),
 ('html', 'python', 'ruby', 'xml'),
 ('.net', 'c#', 'python', 'visual-studio', 'xml'),
 ('asp.net', 'css', 'html', 'javascript', 'performance', 'php'),

In [27]:
from sklearn.externals import joblib
joblib.dump(clf, "model.pkl")
joblib.dump(vectorizer, "Xtransform.pkl")

['Xtransform.pkl']

## Load and use the model

In [28]:
clf = joblib.load("model.pkl")
vectorizer = joblib.load("Xtransform.pkl")
